### Data Preprocessing
To account for the differing rates of atmospheric data, cosmic ray data,
and missing periods of atmospheric data we will need to do the following:
1. Time-align scaler and atmospheric data using timestamps.
2. Remove periods where atmospheric data are unavailable for more than three hours and periods outside
when Scaler data are available.
3. Downsample and interpolate atmospheric data to match the timestamps of the scaler data.
To account for the differing sample rates of the scaler and atmospheric datasets, we will use the average
of the atmospheric data within each 15-minute event-counting window as the feature vector for that target
value.

In [1]:
# Import block from studio 8 will get changed

import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_colwidth", 100)


In [2]:
# Loads into the two csv files, with target being the scalar particle data and weather being weather

targets = pd.read_csv("../Data/scalers.csv")  # Particle count rates
targets = targets.drop(
    columns=["pressure"], errors="ignore"
)  # Drop scaler pressure
weather = pd.read_csv("../Data/weather.csv")  # Atmospheric features

# This drops the pressure column in the scalar data, want to keep the features and targets seperate
targets["time"] = pd.to_numeric(targets["time"])
weather["time"] = pd.to_numeric(weather["time"])

# Makes a new column called "dt" that is the time difference between the data points in target.
targets["dt"] = targets["time"].diff()
# This only keeps data that has "dt" values of 900 seconds or 15 minutes. We are not interested in other data for now
targets_cut = targets[targets["dt"] == 900].reset_index(drop=True)
# Saves the number of targets for later
num_targets = len(targets_cut)
print("Number of target points (scalers):", num_targets)


# Applies a weather cut to only keep data that is within the range of the scalar data after doing the 900 second cut
weather_cut = weather[
    (weather["time"] >= targets_cut["time"].iloc[0])
    & (weather["time"] <= targets_cut["time"].iloc[-1])
].copy()

# This calculates the gaps for the weather data using the same method as for the scalar data
weather_cut["dt"] = weather_cut["time"].diff()
# This is a sanity check to print out the unique gaps in time for the weather data. This should be 300 seconds because the weather is taken with 5 minute intervals
unique_gaps = weather_cut["dt"].dropna().unique()
# This is a snity check to print all the different gaps in weather, this should all be 300
print("Unique time gaps in weather (s):", unique_gaps)


# This resets the index after the cut
weather_cut = weather_cut.reset_index(drop=True)
# This makes a new column called block and assigns each weather data point to a block in groups of three so each block has three data points corresponding to 3, 5 minute intervals
weather_cut["block"] = np.arange(len(weather_cut)) // 3
print(weather_cut.head())

# This groups the weather data by block and calculates the mean weather for each variable for each block and creates a new data_frame called avg_weather.
# It saves the last timestamp as that blocks corresponding time
avg_weather = (
    weather_cut.groupby("block")
    .agg(
        {
            "time": "last",  # take last time of the block
            "temperature": "mean",
            "pressure": "mean",
            "density": "mean",
            "density2HoursBefore": "mean",
        }
    )
    .reset_index(drop=True)
)


# This is a sanity to check to make sure that all the data is sorted by time
targets_cut = targets_cut.sort_values("time").reset_index(drop=True)
avg_weather = avg_weather.sort_values("time").reset_index(drop=True)


# Ensure the data is the same type
avg_weather["time"] = avg_weather["time"].astype(np.int64)

# About to sanity check the time is accurate across data points
avg_weather = avg_weather.rename(columns={"time": "weather_time"})

# This merges the target time data and the weater data on time.
ml_df = pd.merge_asof(
    targets_cut,
    avg_weather,
    left_on="time",
    right_on="weather_time",
    direction="backward",
)

# This calculates the difference in the timestamps
ml_df["time_diff_seconds"] = ml_df["time"] - ml_df["weather_time"]
ml_df["time_diff_minutes"] = ml_df["time_diff_seconds"] / 60


# Prints
print(
    ml_df[
        ["time", "weather_time", "time_diff_seconds", "time_diff_minutes"]
    ].head(10)
)
print("Time difference stats (seconds):")
print(ml_df["time_diff_seconds"].describe())


# Drops the dt columsn because we do not need them
ml_df = ml_df.drop(
    columns=[col for col in ml_df.columns if col == "dt"], errors="ignore"
)

# Removes duplicated time columns
ml_df = ml_df.loc[:, ~ml_df.columns.duplicated()]

# Removes NaN
ml_df = ml_df.dropna().reset_index(drop=True)

print("Final aligned length after dropping NaNs:", len(ml_df))


ml_df


Number of target points (scalers): 453085
Unique time gaps in weather (s): [300.]
         time  temperature  pressure  density  density2HoursBefore     dt  \
0  1112217000        16.79     860.5    1.059                1.034    NaN   
1  1112217300        16.89     860.5    1.059                1.034  300.0   
2  1112217600        16.82     860.5    1.059                1.034  300.0   
3  1112217900        16.62     860.5    1.059                1.034  300.0   
4  1112218200        16.55     860.5    1.059                1.035  300.0   

   block  
0      0  
1      0  
2      0  
3      1  
4      1  
         time  weather_time  time_diff_seconds  time_diff_minutes
0  1112216850           NaN                NaN                NaN
1  1112217750  1.112218e+09              150.0                2.5
2  1112218650  1.112218e+09              150.0                2.5
3  1112219550  1.112219e+09              150.0                2.5
4  1112220450  1.112220e+09              150.0             

,time,rateCorr,arrayFraction,rateUncorr,weather_time,temperature,pressure,density,density2HoursBefore,time_diff_seconds,time_diff_minutes
0,1112217750,188.883,100.0,1921.05,1.112218e+09,16.833333,860.500000,1.059000,1.034000,150.0,2.5
1,1112218650,188.940,100.0,1921.08,1.112218e+09,16.573333,860.500000,1.059000,1.034667,150.0,2.5
2,1112219550,188.841,100.0,1919.44,1.112219e+09,16.496667,860.500000,1.059000,1.035000,150.0,2.5
3,1112220450,188.950,99.9,1919.79,1.112220e+09,16.156667,860.700000,1.059000,1.036667,150.0,2.5
4,1112221350,188.970,100.0,1918.88,1.112221e+09,15.823333,860.800000,1.058667,1.037333,150.0,2.5
...,...,...,...,...,...,...,...,...,...,...,...
453079,1607380650,182.398,98.7,1879.48,1.607380e+09,28.313333,857.133333,1.018000,0.990400,150.0,2.5
453080,1607381550,182.282,98.7,1877.68,1.607381e+09,28.126667,857.100000,1.017000,0.991000,150.0,2.5
453081,1607382450,182.269,98.5,1876.61,1.607382e+09,27.353333,857.266667,1.017000,0.993767,150.0,2.5
453082,1607383350,182.588,98.5,1877.99,1.607383e+09,26.583333,857.466667,1.017000,0.996567,150.0,2.5


In [3]:
ml_df["month"] = pd.to_datetime(ml_df["time"], unit="s").dt.month

ml_df["year"] = pd.to_datetime(ml_df["time"], unit="s").dt.year

ml_df

,time,rateCorr,arrayFraction,rateUncorr,weather_time,temperature,pressure,density,density2HoursBefore,time_diff_seconds,time_diff_minutes,month,year
0,1112217750,188.883,100.0,1921.05,1.112218e+09,16.833333,860.500000,1.059000,1.034000,150.0,2.5,3,2005
1,1112218650,188.940,100.0,1921.08,1.112218e+09,16.573333,860.500000,1.059000,1.034667,150.0,2.5,3,2005
2,1112219550,188.841,100.0,1919.44,1.112219e+09,16.496667,860.500000,1.059000,1.035000,150.0,2.5,3,2005
3,1112220450,188.950,99.9,1919.79,1.112220e+09,16.156667,860.700000,1.059000,1.036667,150.0,2.5,3,2005
4,1112221350,188.970,100.0,1918.88,1.112221e+09,15.823333,860.800000,1.058667,1.037333,150.0,2.5,3,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...
453079,1607380650,182.398,98.7,1879.48,1.607380e+09,28.313333,857.133333,1.018000,0.990400,150.0,2.5,12,2020
453080,1607381550,182.282,98.7,1877.68,1.607381e+09,28.126667,857.100000,1.017000,0.991000,150.0,2.5,12,2020
453081,1607382450,182.269,98.5,1876.61,1.607382e+09,27.353333,857.266667,1.017000,0.993767,150.0,2.5,12,2020
453082,1607383350,182.588,98.5,1877.99,1.607383e+09,26.583333,857.466667,1.017000,0.996567,150.0,2.5,12,2020


In [4]:
data = ml_df[ml_df["year"] > 2012]

data

,time,rateCorr,arrayFraction,rateUncorr,weather_time,temperature,pressure,density,density2HoursBefore,time_diff_seconds,time_diff_minutes,month,year
231261,1356999750,182.835,99.8,1837.12,1.357000e+09,17.333333,864.933333,1.014000,1.037333,150.0,2.5,1,2013
231262,1357000650,182.698,99.9,1832.88,1.357000e+09,16.756667,865.333333,1.014000,1.039667,150.0,2.5,1,2013
231263,1357001550,182.666,99.9,1830.05,1.357001e+09,16.436667,865.733333,1.014000,1.041667,150.0,2.5,1,2013
231264,1357002450,182.604,99.7,1826.28,1.357002e+09,15.740000,866.200000,1.014333,1.044667,150.0,2.5,1,2013
231265,1357003350,182.621,99.7,1823.31,1.357003e+09,14.776667,866.466667,1.015000,1.048333,150.0,2.5,1,2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...
453079,1607380650,182.398,98.7,1879.48,1.607380e+09,28.313333,857.133333,1.018000,0.990400,150.0,2.5,12,2020
453080,1607381550,182.282,98.7,1877.68,1.607381e+09,28.126667,857.100000,1.017000,0.991000,150.0,2.5,12,2020
453081,1607382450,182.269,98.5,1876.61,1.607382e+09,27.353333,857.266667,1.017000,0.993767,150.0,2.5,12,2020
453082,1607383350,182.588,98.5,1877.99,1.607383e+09,26.583333,857.466667,1.017000,0.996567,150.0,2.5,12,2020


In [5]:
# assume your dataframe is called df and has a 'year' column

encoder = OneHotEncoder(sparse_output=False, dtype=int).set_output(
    transform="pandas"
)  # easier to work with

# Fit + transform
year_df = encoder.fit_transform(data[["year"]])

# Drop original year column and concat encoded columns
data = pd.concat([data, year_df], axis=1)

data

,time,rateCorr,arrayFraction,rateUncorr,weather_time,temperature,pressure,density,density2HoursBefore,time_diff_seconds,time_diff_minutes,month,year,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020
231261,1356999750,182.835,99.8,1837.12,1.357000e+09,17.333333,864.933333,1.014000,1.037333,150.0,2.5,1,2013,1,0,0,0,0,0,0,0
231262,1357000650,182.698,99.9,1832.88,1.357000e+09,16.756667,865.333333,1.014000,1.039667,150.0,2.5,1,2013,1,0,0,0,0,0,0,0
231263,1357001550,182.666,99.9,1830.05,1.357001e+09,16.436667,865.733333,1.014000,1.041667,150.0,2.5,1,2013,1,0,0,0,0,0,0,0
231264,1357002450,182.604,99.7,1826.28,1.357002e+09,15.740000,866.200000,1.014333,1.044667,150.0,2.5,1,2013,1,0,0,0,0,0,0,0
231265,1357003350,182.621,99.7,1823.31,1.357003e+09,14.776667,866.466667,1.015000,1.048333,150.0,2.5,1,2013,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453079,1607380650,182.398,98.7,1879.48,1.607380e+09,28.313333,857.133333,1.018000,0.990400,150.0,2.5,12,2020,0,0,0,0,0,0,0,1
453080,1607381550,182.282,98.7,1877.68,1.607381e+09,28.126667,857.100000,1.017000,0.991000,150.0,2.5,12,2020,0,0,0,0,0,0,0,1
453081,1607382450,182.269,98.5,1876.61,1.607382e+09,27.353333,857.266667,1.017000,0.993767,150.0,2.5,12,2020,0,0,0,0,0,0,0,1
453082,1607383350,182.588,98.5,1877.99,1.607383e+09,26.583333,857.466667,1.017000,0.996567,150.0,2.5,12,2020,0,0,0,0,0,0,0,1


In [7]:
data.to_csv("../Data/prepro_data.csv", index=False)